In [1]:
"""##Importing the libraries"""

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from scipy.sparse import *

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
"""## Importing the dataset"""
dataset = pd.read_csv('/content/drive/MyDrive/PRNG/4-mil-test-set.csv')

In [ ]:
#Understanding your dataset first 
dataset.head()
dataset.dtypes == 'object'

id        False
result    False
hash       True
dtype: bool

In [ ]:
#separate the categorical variables from the numerical variables
num_vars = dataset.columns[dataset.dtypes != 'object']
cat_vars = dataset.columns[dataset.dtypes != 'object']

In [ ]:
print(num_vars) 
print(cat_vars)


Index(['id', 'result'], dtype='object')
Index(['id', 'result'], dtype='object')


In [ ]:
#finding the missing values in the dataset and sorting them in order
dataset.isnull().sum().sort_values(ascending = False)

hash      0
result    0
id        0
dtype: int64

In [4]:
#dataset = dataset[dataset.result < 2000]
#dataset = dataset[dataset.id < 20000]
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [5]:
"""## Encoding categorical data"""
"""# Label Encoding the "hash" column"""

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [6]:
"""## Splitting the dataset into the Training set and Test set"""

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
"""## Feature Scaling"""
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (1, 1000))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
"""## Preparing the training set and the test set"""
X_train_df = pd.DataFrame(X_train)
y_train_df = pd.DataFrame(y_train)
X_train_df[2] = y_train_df[0]
training_set = np.array(X_train_df, dtype='int')


X_test_df = pd.DataFrame(X_test)
y_test_df = pd.DataFrame(y_test)
X_test_df[2] = y_test_df[0]
test_set = np.array(X_test_df, dtype='int')

In [9]:
"""## Getting the number of ids and results"""

nb_id = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_results = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [10]:
"""## Converting the data into an array with id in lines and result in columns"""

def convert(data):
  new_data = []
  for id_users in range(1, nb_id + 1):
    id_result = data[:, 1] [data[:, 0] == id_users]
    id_hash = data[:, 2] [data[:, 0] == id_users]
    sha_hash = np.zeros(nb_results)
    sha_hash[id_result - 1] = id_hash
    new_data.append(list(sha_hash))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [11]:
"""## Converting the data into Torch tensors"""

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [12]:
"""## Creating the architecture of the Neural Network"""

class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_results, 30)
        self.fc2 = nn.Linear(30, 20)
        self.fc3 = nn.Linear(20, 10)
        self.fc4 = nn.Linear(10, 20)
        self.fc5 = nn.Linear(20, 30)
        self.fc6 = nn.Linear(30, nb_results)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.activation(self.fc4(x))
        x = self.activation(self.fc5(x))
        x = self.fc6(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [14]:
"""## Training the SAE"""

nb_epoch = 2000
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_id):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_results/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(2208992.7500)
epoch: 2loss: tensor(2208730.5000)
epoch: 3loss: tensor(2208471.5000)
epoch: 4loss: tensor(2208211.5000)
epoch: 5loss: tensor(2207954.2500)
epoch: 6loss: tensor(2207698.)
epoch: 7loss: tensor(2207440.2500)
epoch: 8loss: tensor(2207182.7500)
epoch: 9loss: tensor(2206925.2500)
epoch: 10loss: tensor(2206669.7500)
epoch: 11loss: tensor(2206412.)
epoch: 12loss: tensor(2206154.2500)
epoch: 13loss: tensor(2205896.7500)
epoch: 14loss: tensor(2205639.7500)
epoch: 15loss: tensor(2205383.2500)
epoch: 16loss: tensor(2205126.7500)
epoch: 17loss: tensor(2204871.)
epoch: 18loss: tensor(2204615.2500)
epoch: 19loss: tensor(2204360.2500)
epoch: 20loss: tensor(2204103.5000)
epoch: 21loss: tensor(2203848.)
epoch: 22loss: tensor(2203591.7500)
epoch: 23loss: tensor(2203336.5000)
epoch: 24loss: tensor(2203078.5000)
epoch: 25loss: tensor(2202822.2500)
epoch: 26loss: tensor(2202565.5000)
epoch: 27loss: tensor(2202308.2500)
epoch: 28loss: tensor(2202051.7500)
epoch: 29loss: te

KeyboardInterrupt: ignored

In [ ]:
"""## Testing the SAE"""

test_loss = 0
s = 0.
for id_user in range(nb_id):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_results/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(446.6451)


In [ ]:
"""## Extracting the title of the id and result from the dataset"""
id_title = dataset.iloc[:nb_results, 0:1]
result_title = dataset.iloc[:nb_results, 1:-1]

In [ ]:
"""## Choosing a User by the Test Set and taking the whole list of Hashes of that User"""
user_id = 101
user_hash = training_set.data.numpy()[user_id - 1, :].reshape(-1,1)
user_target = test_set.data.numpy()[user_id, :].reshape(-1,1)

In [ ]:
"""## Making the Predictions using the Input"""
user_input = Variable(training_set[user_id]).unsqueeze(0)
predicted = sae(user_input)
predicted = predicted.data.numpy().reshape(-1,1)
predicted

array([[ 8.42002583e+00],
       [ 1.85222378e+01],
       [ 5.82904196e+00],
       [-7.90211773e+00],
       [ 2.93536072e+01],
       [ 1.82029285e+01],
       [ 2.84780502e-01],
       [-2.13293600e+00],
       [ 8.75118065e+00],
       [ 2.12298565e+01],
       [ 4.03245306e+00],
       [ 1.17515888e+01],
       [ 1.25064745e+01],
       [-5.02648652e-01],
       [ 2.46538239e+01],
       [-4.05574570e+01],
       [-1.71374536e+00],
       [ 1.84427142e+00],
       [ 1.35299349e+00],
       [ 2.57099838e+01],
       [-4.08914804e+00],
       [-6.50874138e-01],
       [ 1.02340450e+01],
       [ 9.22421360e+00],
       [ 2.05753651e+01],
       [ 3.05451221e+01],
       [ 5.41918421e+00],
       [ 5.29879379e+00],
       [ 9.44412994e+00],
       [ 5.79256105e+00],
       [-1.40931916e+00],
       [ 7.87655450e-03],
       [ 2.67270517e+00],
       [ 2.97969079e+00],
       [ 7.14318943e+00],
       [-3.42463493e+00],
       [ 7.19246817e+00],
       [ 6.74249649e+00],
       [ 3.7

In [ ]:
"""## Combining all the info into one dataframe"""
hash_array = np.hstack([id_title, user_target, predicted])
hash_array = hash_array[hash_array[:, 1] > 0]
hash_df = pd.DataFrame(data=hash_array, columns=['id', 'Target Hash', 'Predicted'])
hash_df

,id,Target Result,Predicted
0,1.0,15.0,8.420026
1,2.0,0.0,18.522238
2,3.0,1.0,5.829042
3,4.0,0.0,-7.902118
4,5.0,0.0,29.353607
...,...,...,...
995,996.0,1.0,21.348196
996,997.0,1.0,-3.809019
997,998.0,0.0,8.654700
998,999.0,1.0,25.848877


In [ ]:
dataset['result'].iloc[-1]

1.33